In [1]:
import sys
import os

import ccxt
import backtesting as bt
from backtesting import Backtest, Strategy
import talib
import pandas_ta as ta

import pandas as pd
import numpy as np

In [2]:
def get_data(symbol, timeframe, limit):
    exchange = ccxt.kraken()
    exchange.load_markets()

    data = exchange.fetch_ohlcv('BTC/USDT', timeframe=timeframe, limit=limit)
    df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']).rename(columns={
        'timestamp': 'timestamp',
        'open': 'Open',
        'high': 'High',
        'low': 'Low',
        'close': 'Close',
        'volume': 'Volume'
    })
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

In [3]:
data = get_data('BTC/USDT', '1h', 1000)

In [4]:
data

,Open,High,Low,Close,Volume
timestamp,,,,,
2024-04-27 03:00:00,62947.0,63136.9,62841.0,63070.6,3.890570
2024-04-27 04:00:00,63049.1,63161.5,62957.3,63013.3,0.562865
2024-04-27 05:00:00,63029.9,63073.0,62857.5,62981.3,0.806780
2024-04-27 06:00:00,62981.2,63110.7,62890.0,63005.0,1.887178
2024-04-27 07:00:00,63006.9,63139.2,62950.6,62965.5,6.454611
...,...,...,...,...,...
2024-05-26 22:00:00,68494.7,68693.6,68481.9,68595.6,3.915592
2024-05-26 23:00:00,68559.7,68559.8,68356.7,68531.7,1.545142
2024-05-27 00:00:00,68503.1,68762.1,68476.7,68756.5,1.062076


# EMA strategy

In [5]:
class EMA_strategy(Strategy):
    ############################ Parameters ####################################
    ema_a = 36
    ema_b = 60

    ############################ Utilities ####################################
    # For logging
    def log(self, txt, dt=None):
        dt = dt or self.data.index[-1]
        # print(f'{dt.strftime("%Y-%m-%d %H:%M:%S")}: {txt}')
    
    ############################ Strategy: Calculate the indicators ####################################

    def init(self):
        
        # print(f"BB_SMA: {self.BB_SMA}, BB_STD: {self.BB_STD}, BB_MAX_BANDWIDTH: {self.BB_MAX_BANDWIDTH}")
        # print(f"min_volatility: {self.min_volatility}, max_buy_perc: {self.max_buy_perc}, min_sell_perc: {self.min_sell_perc}")
        
        self.line_a = self.I(lambda df, length: df.ta.ema(close = df['Close'], length=length), self.data.df, self.ema_a, name='line_a')
        self.line_b = self.I(lambda df, length: df.ta.ema(close = df['Close'], length=length), self.data.df, self.ema_b, name='line_b')
        
        self.buy_signal = self.I(lambda line_a, line_b: (line_a > line_b), self.line_a, self.line_b, name='buy_signal')
        self.sell_signal = self.I(lambda line_a, line_b: (line_a < line_b), self.line_a, self.line_b, name='sell_signal')
    def next(self):
        
        self.log(f"Close: {self.data.Close[-1]}, position: {self.position.size}, cash: {self._broker.margin_available}")
        
        if self._broker.orders:
            return
        
        if self.position.size == 0:
            if self.buy_signal[-1]:
                self.log("BUY")
                self.buy()
                
            elif self.sell_signal[-1]:
                self.log("SELL")
                self.sell()
                
        elif self.position.size > 0:
            if self.sell_signal[-1]:
                self.log("Close BUY")
                self.position.close()
                
                self.log("SELL")
                self.sell()
                
        elif self.position.size < 0:
            if self.buy_signal[-1]:
                self.log("Close SELL")
                self.position.close()
                
                self.log("BUY")
                self.buy()
                

In [6]:
bt = Backtest(
    data, 
    EMA_strategy,
    commission=0.00075, 
    cash=100000, 
    )

strat = bt.run() 
bt.plot()

Row(id='1566', ...)

In [7]:
pd.DataFrame(strat)

,0
Start,2024-04-27 03:00:00
End,2024-05-27 02:00:00
Duration,29 days 23:00:00
Exposure Time [%],91.527778
Equity Final [$],98251.955875
Equity Peak [$],106107.023225
Return [%],-1.748044
Buy & Hold Return [%],9.669799
Return (Ann.) [%],-18.749908
Volatility (Ann.) [%],27.582868


## Optimization

In [8]:
import multiprocessing as mp
mp.set_start_method('fork') 

In [9]:
bt = Backtest(
    data, 
    EMA_strategy,
    commission=0.00075, 
    cash=100000, 
    )

stats = bt.optimize(
                    ema_a=range(5, 70, 1), # 5-70 
                    ema_b=range(5, 70, 1), # 5-70
                    constraint=lambda param: param.ema_a < param.ema_b,
                    maximize='Sharpe Ratio',
                    max_tries=500,
                    random_state=1) 

pd.DataFrame(stats)
stats._strategy

/Users/cholian/anaconda3/envs/tokenmetrics/lib/python3.10/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 501 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/11 [00:00<?, ?it/s]

<Strategy EMA_strategy(ema_a=13,ema_b=17)>

In [10]:
bt = Backtest(
    data, 
    EMA_strategy,
    commission=0.00075, 
    cash=100000, 
    )

strat = bt.run(ema_a=13, ema_b=17)
bt.plot()

Row(id='2784', ...)

In [11]:
strat

Start                     2024-04-27 03:00:00
End                       2024-05-27 02:00:00
Duration                     29 days 23:00:00
Exposure Time [%]                        97.5
Equity Final [$]                 104863.06085
Equity Peak [$]                 108529.646825
Return [%]                           4.863061
Buy & Hold Return [%]                9.669799
Return (Ann.) [%]                   71.342725
Volatility (Ann.) [%]               51.684928
Sharpe Ratio                         1.380339
Sortino Ratio                        4.296442
Calmar Ratio                        10.777848
Max. Drawdown [%]                   -6.619385
Avg. Drawdown [%]                   -1.386133
Max. Drawdown Duration       14 days 15:00:00
Avg. Drawdown Duration        1 days 23:00:00
# Trades                                   26
Win Rate [%]                        30.769231
Best Trade [%]                       7.980765
Worst Trade [%]                     -2.232252
Avg. Trade [%]                    

In [12]:
strat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,18,45,63324.858050,63106.3,-218.558050,-0.003451,2024-04-27 21:00:00,2024-04-29 00:00:00,1 days 03:00:00
1,-1,45,67,63058.970275,63087.8,-28.829725,-0.000457,2024-04-29 00:00:00,2024-04-29 22:00:00,0 days 22:00:00
2,1,67,78,63135.115850,61868.4,-1266.715850,-0.020064,2024-04-29 22:00:00,2024-04-30 09:00:00,0 days 11:00:00
3,-1,78,130,61821.998700,58558.0,3263.998700,0.052797,2024-04-30 09:00:00,2024-05-02 13:00:00,2 days 04:00:00
4,1,130,228,58601.918500,63278.8,4676.881500,0.079808,2024-05-02 13:00:00,2024-05-06 15:00:00,4 days 02:00:00
5,-1,228,246,63231.340900,64233.2,-1001.859100,-0.015844,2024-05-06 15:00:00,2024-05-07 09:00:00,0 days 18:00:00
6,1,246,251,64281.374900,63197.2,-1084.174900,-0.016866,2024-05-07 09:00:00,2024-05-07 14:00:00,0 days 05:00:00
7,-1,251,252,63149.802100,64033.1,-883.297900,-0.013987,2024-05-07 14:00:00,2024-05-07 15:00:00,0 days 01:00:00
8,1,252,255,64081.124825,63380.4,-700.724825,-0.010935,2024-05-07 15:00:00,2024-05-07 18:00:00,0 days 03:00:00
9,-1,255,304,63332.864700,62257.8,1075.064700,0.016975,2024-05-07 18:00:00,2024-05-09 19:00:00,2 days 01:00:00


In [17]:
strat['_equity_curve']

,Equity,DrawdownPct,DrawdownDuration
2024-04-27 03:00:00,100000.00000,0.000000,NaT
2024-04-27 04:00:00,100000.00000,0.000000,NaT
2024-04-27 05:00:00,100000.00000,0.000000,NaT
2024-04-27 06:00:00,100000.00000,0.000000,NaT
2024-04-27 07:00:00,100000.00000,0.000000,NaT
...,...,...,...
2024-05-26 22:00:00,105346.56085,0.029329,NaT
2024-05-26 23:00:00,105410.46085,0.028740,NaT
2024-05-27 00:00:00,105185.66085,0.030812,NaT
2024-05-27 01:00:00,104802.16085,0.034345,NaT
